[Home](../../README.md)

### Data Wrangling

This is a demonstration of data wrangling using [Pandas](https://pandas.pydata.org/) the library for data analysis and manipulation.

This Jupyter Notepad demonstrates different processes you can apply to your data to prepare it for feature engineering and model training. For this demonstration we will wrangle the poker hand data set you previewed in the last Jupyter Notebook.

> [!Note]
> None of these processes are destructive to the source CSV as long as you save the modified data to a new CSV.

#### Load the required dependencies

In [3]:
# Import frameworks
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

####  Store the data as a local variable

The data frame is a Pandas object that structures your tabular data into an appropriate format. It loads the complete data in memory so it is now ready for preprocessing.

In [4]:
data_frame = pd.read_csv("poker_hand_dataset.csv")

#### Dealing with null values

Null values during data analysis can cause runtime errors and unexpected results. It is important to identify null values and deal with them appropriately before training a model.

The `isnull().sum()` method call returns the null values in any column.

In [5]:
data_frame.isnull().sum()

Suit of Card 1    0
Suit of Card 2    0
Suit of Card 3    0
Suit of Card 4    0
Suit of Card 5    0
Rank of Card 1    0
Rank of Card 2    0
Rank of Card 3    0
Rank of Card 4    0
Rank of Card 5    0
Poker Hand        0
dtype: int64

### Null Data Check

Shown below is the result of checking for null values in the dataset. As we can see, there are no null values in any of the columns, indicating that the dataset is clean and ready for further processing.

In [6]:
# Remove Null values
data_frame.isnull().sum()

Suit of Card 1    0
Suit of Card 2    0
Suit of Card 3    0
Suit of Card 4    0
Suit of Card 5    0
Rank of Card 1    0
Rank of Card 2    0
Rank of Card 3    0
Rank of Card 4    0
Rank of Card 5    0
Poker Hand        0
dtype: int64

#### Remove Duplicates

Duplicate data can have detrimental effects on your machine learning models and outcomes, such as reducing data diversity and representativeness, which can lead to overfitting or biased models.

The `duplicated().sum()` method call returns the count of duplicate rows in the data frame.

In [7]:
data_frame.duplicated().sum()

np.int64(2)

The `drop_duplicates()` method call can be then stored back onto the data_frame variable removing the duplicates.

In [8]:
data_frame = data_frame.drop_duplicates()
data_frame.duplicated().sum()

np.int64(0)

#### Replace Data

We can run a lambda function on a column to modify its values. For example, if we wanted to standardize the format of card suits or ranks, we could convert them to lowercase or encode them numerically. To run a function over a complete column, we can use the `apply` method, which iterates over each row and modifies the values.

In [9]:
# Standardize the format of card suits in the 'Suit of Card 1' column
data_frame['Suit of Card 1'] = data_frame['Suit of Card 1'].apply(lambda x: str(x).lower())

# Preview the changes
data_frame['Suit of Card 1'].head()


0    4
1    4
2    2
3    3
4    1
Name: Suit of Card 1, dtype: object

We can check that there are no data entry errors by the `unique()` method call.

In [12]:
print("Unique values before conversion:", data_frame['Suit of Card 1'].unique())
print("Data type:", data_frame['Suit of Card 1'].dtype)


Unique values before conversion: ['4' '2' '3' '1']
Data type: object


In [17]:
# Define the suit mapping
suit_mapping = {
    1: 'Clubs',
    2: 'Diamonds',
    3: 'Hearts',
    4: 'Spades'
}

# Ensure that 'Suit of Card 1' is of integer type
data_frame['Suit of Card 1'] = data_frame['Suit of Card 1'].astype(int)

# Map the numeric suits to text suits
data_frame['Suit of Card 1 Text'] = data_frame['Suit of Card 1'].map(lambda x: suit_mapping.get(x, 'unknown'))

# Check the results
print("Mapped Text Suits:", data_frame['Suit of Card 1 Text'].unique())

Mapped Text Suits: ['Spades' 'Diamonds' 'Hearts' 'Clubs']


#### Remove outliers

Outliers can skew your analysis on numerical columns, and it is important to remove them. We can use the 25th and 75th quartile on numerical data, to get the inter-quartile range. This allows us to estimate an acceptable range, and we can then filter out any values outside this range. Mathematically, outliers are values occurring outside 1.5 times the interquartile range (IQR) from the first quartile (Q1) or third quartile (Q3).

In [18]:
# Get the interquartile range (IQR) for the 'Rank of Card 1' column
print(data_frame['Rank of Card 1'].describe())

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = data_frame['Rank of Card 1'].quantile(0.25)
Q3 = data_frame['Rank of Card 1'].quantile(0.75)

# Calculate the IQR
IQR = Q3 - Q1

# Identify outliers
print(f'Outliers are a Rank of Card 1 above {Q3 + IQR * 1.5} or below {Q1 - IQR * 1.5}')

count    49998.000000
mean         7.997800
std          3.736083
min          2.000000
25%          5.000000
50%          8.000000
75%         11.000000
max         14.000000
Name: Rank of Card 1, dtype: float64
Outliers are a Rank of Card 1 above 20.0 or below -4.0


In [19]:

# Filter rows within the acceptable range for 'Poker Hand'
data_frame = data_frame[(data_frame['Poker Hand'] >= Q1 - 1.5 * IQR) & (data_frame['Poker Hand'] <= Q3 + 1.5 * IQR)]

# Display the descriptive statistics for the filtered data
print(data_frame['Poker Hand'].describe())

count    49998.000000
mean         1.621705
std          1.487787
min          0.000000
25%          1.000000
50%          1.000000
75%          3.000000
max          9.000000
Name: Poker Hand, dtype: float64


#### Checking if Columns Are Already Scaled

Before scaling the features, it is important to check if they are already scaled. Scaled features typically have values within a specific range, such as 0 to 1 for Min-Max scaling.

To check if a column is already scaled:
1. Inspect the minimum and maximum values of the column.
2. If the values are between 0 and 1, the column is likely already scaled.
3. If the values are outside this range, scaling may be required.



In [23]:
# Define rank and suit columns
rank_columns = ['Rank of Card 1', 'Rank of Card 2', 'Rank of Card 3', 'Rank of Card 4', 'Rank of Card 5']
suit_columns = ['Suit of Card 1', 'Suit of Card 2', 'Suit of Card 3', 'Suit of Card 4', 'Suit of Card 5']

# Filter out invalid rank values
for column in rank_columns:
    data_frame = data_frame[data_frame[column].between(1, 13)]

# Scale the rank columns
MIN_RANK = 1
MAX_RANK = 13
for column in rank_columns:
    data_frame[column] = [(X - MIN_RANK) / (MAX_RANK - MIN_RANK) for X in data_frame[column]]

# Scale the suit columns (optional)
MIN_SUIT = 1
MAX_SUIT = 4
for column in suit_columns:
    data_frame[column] = [(X - MIN_SUIT) / (MAX_SUIT - MIN_SUIT) for X in data_frame[column]]

# Check the ranges again
print("Rechecking ranges for rank columns:")
for column in rank_columns:
    print(f"{column}: Min = {data_frame[column].min()}, Max = {data_frame[column].max()}")

print("\nRechecking ranges for suit columns:")
for column in suit_columns:
    print(f"{column}: Min = {data_frame[column].min()}, Max = {data_frame[column].max()}")

Rechecking ranges for rank columns:
Rank of Card 1: Min = nan, Max = nan
Rank of Card 2: Min = nan, Max = nan
Rank of Card 3: Min = nan, Max = nan
Rank of Card 4: Min = nan, Max = nan
Rank of Card 5: Min = nan, Max = nan

Rechecking ranges for suit columns:
Suit of Card 1: Min = nan, Max = nan
Suit of Card 2: Min = nan, Max = nan
Suit of Card 3: Min = nan, Max = nan
Suit of Card 4: Min = nan, Max = nan
Suit of Card 5: Min = nan, Max = nan


> [!important]
> You need to save the calculations for each dataset you scale for scaling new values for prediction.

#### Save the wrangled data to CSV

In [24]:
data_frame.to_csv('poker_hand_dataset_wrangled_data.csv', index=False)